In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
from tkinter import Tk, Button, filedialog
%matplotlib inline

def display():
    global FICHIER, FENETRE
    FICHIER = filedialog.askopenfilename(filetypes=(("all files", "*.txt"),))
    FENETRE.destroy()

FENETRE = Tk() 
button = Button(FENETRE, text='Choisissez un fichier :', command=display)
button.pack()
FENETRE.mainloop() 

"""récupère les données du txt choisi"""
data = pd.read_csv(FICHIER, sep="\t", index_col=0, skiprows=2)

"""calcul des tangentes plastiques et post-plastiques"""
a_plast = (data.loc[15,"Y"] - data.loc[5,"Y"]) / 10
b_plast = data.loc[10,"Y"] - a_plast * 10
a_post = (data.loc[45,"Y"] - data.loc[35,"Y"]) / 10
b_post = data.loc[40,"Y"] - a_post * 40
tangentes = {"plastique":lambda x : a_plast * x + b_plast, "post-plastique":lambda x : a_post * x + b_post}

"""création et paramètres de la zone de graphique"""
fig = plt.figure(figsize=(10,4.5))
ax = plt.axes()
plt.title('Détermination de la contrainte de sollicitation à utiliser en fatigue cyclique', fontsize=14, pad=10)
ax.set(xlim=(0, 60), ylim=(0, np.ceil(data['Y'].max() / 10) * 10))
ax.set_xlabel('Allongement (%)', labelpad=10)
ax.set_ylabel('Contrainte (MPa)', labelpad=15)

"""tracé de la courbe de traction et des deux tangentes"""
plt.plot(data.loc[0:70], color='b', linewidth=3)
plt.plot(range(0,40), [tangentes['plastique'](x) for x in range(0,40)], color='r')
plt.plot(range(20,55), [tangentes['post-plastique'](x) for x in range(20,55)], color='g')

"""calcul du point d'intersection des tangentes et affichage du point avec la valeur de contrainte"""
xi = (b_plast - b_post) / (a_post - a_plast)
yi = tangentes['plastique'](xi)
plt.text(yi + 2, xi - 5, f"Contrainte = {round(yi,1)} MPa", fontsize=12)
plt.plot(np.linspace(0, xi, 30), np.repeat(yi, 30), color='k', linestyle='dashed')
plt.plot(np.repeat(xi, 30), np.linspace(0, yi, 30), color='k', linestyle='dashed')
plt.scatter(xi, yi, color='k');